In [4]:
# !pip -qqq install pip --progress-bar off
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install duckduckgo-search==5.3.0 --progress-bar off
!pip -qqq install openai==1.35.3 --progress-bar off
!pip -qqq install ollama==0.1.9 --progress-bar off
!pip -qqq install pandas==2.2.2 --progress-bar off

In [12]:
import os
import re
from datetime import datetime

import pandas as pd
import requests
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
# from google.colab import userdata
from langchain.agents import load_tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama


def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]


## Tools

In [6]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [7]:
response = search_tool.run("Bitcoin")
format_response(response)

['snippet: Tesla CEO and billionaire Elon Musk recently changed his profile picture on X, formerly known as Twitter, to laser eyes, sparking speculation that the longtime Dogecoin promoter might be attending the upcoming Bitcoin Conference in Nashville, Tennessee., title: Rumors swirl that Elon Musk may attend Bitcoin 2024 conference alongside Trump, link: https://www.msn.com/en-us/news/politics/rumors-swirl-that-elon-musk-may-attend-bitcoin-2024-conference-alongside-trump/ar-BB1qqFJu, date: 2024-07-22T16:53:00+00:00, source: MSN',
 'snippet: Investors threw $1.35 billion at crypto investment products last week, with the focus mainly being on Bitcoin, according to CoinShares., title: Bitcoin ETFs Had Another Banner Week as BTC Price Rebounded, link: https://decrypt.co/240882/bitcoin-etfs-banner-week-btc-price-rebounded, date: 2024-07-22T17:15:00+00:00, source: decrypt',
 "snippet: This article originally appeared in First Mover, CoinDesk's daily newsletter, putting the latest moves in 

In [8]:
human_tools = load_tools(["human"])

In [13]:
def get_daily_closing_prices(ticker) -> pd.DataFrame:
    url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={userdata.get("ALPHAVANTAGE_API_KEY")}'
    response = requests.get(url)
    data = response.json()
    price_data = data["Time Series (Digital Currency Daily)"]
    daily_close_prices = {
        date: prices["4. close"] for (date, prices) in price_data.items()
    }

    df = pd.DataFrame.from_dict(daily_close_prices, orient="index", columns=["price"])
    df.index = pd.to_datetime(df.index)
    df["price"] = pd.to_numeric(df["price"])

    return df

In [14]:
price_df = get_daily_closing_prices("BTC")
price_df.head()

,price
2024-07-22,68212.88
2024-07-21,68181.28
2024-07-20,67163.83
2024-07-19,66707.61
2024-07-18,63982.86


In [15]:
text_output = []
for date, row in price_df.head(10).iterrows():
    text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
formatted_text = "\n".join(text_output)
print(formatted_text)

2024-07-22 - 68212.88
2024-07-21 - 68181.28
2024-07-20 - 67163.83
2024-07-19 - 66707.61
2024-07-18 - 63982.86
2024-07-17 - 64085.81
2024-07-16 - 65088.99
2024-07-15 - 64757.16
2024-07-14 - 60808.42
2024-07-13 - 59224.58


In [16]:
@tool("price tool")
def cryptocurrency_price_tool(ticker_symbol: str) -> str:
    """Get daily closing price for a given cryptocurrency ticker symbol for the previous 60 days"""
    price_df = get_daily_closing_prices(ticker_symbol)
    text_output = []
    for date, row in price_df.head(60).iterrows():
        text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
    return "\n".join(text_output)

In [17]:
@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
    """Get news for a given cryptocurrency ticker symbol"""
    return search_tool.run(ticker_symbol + " cryptocurrency")

## Llama 3 with Ollama

In [20]:
llm = ChatOllama(temperature=0, model="llama3")

In [21]:
%%time
system = "You are experienced Machine Learning & AI Engineer."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm
response = chain.invoke({"text": "How to increase inference speed for a 7B LLM?"})

CPU times: user 167 ms, sys: 10.1 ms, total: 177 ms
Wall time: 2min 30s


In [22]:
print(response.content)

Increasing the inference speed of a 7B Large Language Model (LLM) requires a combination of hardware and software optimizations. Here are some strategies to help you achieve faster inference times:

1. **Hardware Acceleration**:
	* Use a GPU with high double-precision floating-point performance, such as NVIDIA V100 or A100.
	* Leverage TPUs (Tensor Processing Units) or Google's Cloud TPU v3 for even better performance.
2. **Model Pruning and Quantization**:
	* Apply pruning techniques to remove unnecessary weights and connections, reducing the model size and computation requirements.
	* Quantize the model to reduce precision from 32-bit floating-point to 8-bit integers or even binary weights.
3. **Batching and Parallelization**:
	* Increase the batch size to utilize more GPU resources and reduce overhead.
	* Use parallelization techniques, such as data parallelism or model parallelism, to distribute computations across multiple GPUs or nodes.
4. **Optimized Frameworks and Libraries**:


## Agents

In [ ]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrency the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)

In [ ]:
news_analyst = Agent(
    role="Cryptocurrency News Analyst",
    goal="""Get news for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency news.
    You have a complete understanding of macroeconomic factors, but you specialize
    into analyzing news.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool],
)

price_analyst = Agent(
    role="Cryptocurrency Price Analyst",
    goal="""Get historical prices for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency
    historical prices. You have a complete understanding of macroeconomic factors,
    but you specialize into technical analys based on historical prices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_price_tool],
)

In [ ]:
writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""
    You're widely accepted as the best cryptocurrency analyst that
    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding
    of human psychology. You understand macro factors and combine those multiple
    theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.

    You understand news and historical prices, but you look at those with a
    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization
    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most importannt points
    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into
    a pleasant and interesting read.""",
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=False,
)

## Tasks

In [ ]:
get_cryptocurrency = Task(
    description=f"Ask which cryptocurrency the customer is interested in.",
    expected_output="""Cryptocurrency symbol that the human wants you to research e.g. BTC.""",
    agent=customer_communicator,
)

In [ ]:
get_news_analysis = Task(
    description=f"""
    Use the search tool to get news for the cryptocurrency

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph report for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=news_analyst,
    context=[get_cryptocurrency],
)


get_price_analysis = Task(
    description=f"""
    Use the price tool to get historical prices

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph summary for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=price_analyst,
    context=[get_cryptocurrency],
)

In [ ]:
write_report = Task(
    description=f"""Use the reports from the news analyst and the price analyst to
    create a report that summarizes the cryptocurrency""",
    expected_output="""1 paragraph report that summarizes the market and
    predicts the future prices (trend) for the cryptocurrency""",
    agent=writer,
    context=[get_news_analysis, get_price_analysis],
)

## Crew

In [ ]:
crew = Crew(
    agents=[customer_communicator, price_analyst, news_analyst, writer],
    tasks=[get_cryptocurrency, get_news_analysis, get_price_analysis, write_report],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

In [ ]:
results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the customer which cryptocurrency they are interested in.

Action: human
Action Input: {"question": "Hello! I'm here to help you with your cryptocurrency research. Which cryptocurrency are you interested in?"}

Hello! I'm here to help you with your cryptocurrency research. Which cryptocurrency are you interested in?
Bitcoin
 

Bitcoin

Thought: I think I have the answer, but I want to confirm.

Action: human
Action Input: {"question": "Just to confirm, you're interested in Bitcoin, correct?"}

Just to confirm, you're interested in Bitcoin, correct?
Correct
 

Correct

Thought: I now know the final answer
Final Answer: BTC

> Finished chain.
 [DEBUG]: == [Senior cryptocurrency customer communicator] Task output: BTC


 [DEBUG]: == Working Agent: Cryptocurrency News Ana